# Bootstrapping

In statistics, it is not always the case that you will have knowledge of the exact, or even approximate probability distributions for your data. In this case, its extremely difficult to just write down a likelihood and proceed to determine estimates, confidence/credible intervals for various parameters that you might be interested in. 

The method of bootstrapping instead, only relies on re-sampling from your existing dataset. Suppose you have a dataset $X_{1},X_{2},...,X_{N}$ of random, *independant observations*. Then you produce a bootstrap sample by randomly selecting $N$ of the observations, replacing them each time. From multiple such samples, you can calculate a quantity (such as an estimator) and use them to determine properties of the distribution of that quantity - eg their variance. 

We can use python's `random.choice` module for this.

In [ ]:
import random

 As an example, picking 6  random elements out of a list of 10 integers is as simple as,

In [ ]:
random.choices([1,4,2,5,6,3,8,7,9,10],k=6)

Note that the `choices` function samples *with replacement* (so we can see the same element more than once in the returned list). To sample *without replacement* in python, we can use `random.sample` instead. 

Now let's generate some random data from our example of a normal distribution with parameters $\mu=3,\sigma=1.5$.  We'll generate 50 observations using the `random` module from numpy. 

In [ ]:
import numpy
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})

from scipy.stats import norm

mu_true = mu = 3.0
sigma_true = sigma = 1.5
observations = numpy.random.normal(mu,sigma,50)
print(observations)

Remember  that the sample mean is given by, 

$$
\bar{x}=\frac{1}{N}\sum_{i=1}^{N}X_{i}
$$

We can estimate the distribution of the mean by using bootstrap samples. Select 50 observations (replacing each time) and in each case calculate the mean from that bootstrap sample. We'll keep track of the values calulated and plot a histogram of them.

In [ ]:
def sample_mean(data): 
    return numpy.mean(data)

nsamples = 10000 
means = []

for s in range(nsamples) : 
    bootstrap = random.choices(observations,k=50)
    means.append(sample_mean(bootstrap))

plt.hist(means,bins=100)
plt.show()

Now comparing to the mean of the original observations, we can see that the mean of the distribution is indeed similar to the estimate from the observations. 

In [2]:
print(sample_mean(observations))
print(sample_mean(means))

NameError: name 'sample_mean' is not defined

From the standard deviation of the means obtained from the bootstrap samples, we can get some idea about the *uncertainty on the mean estimated from the sample mean of the observations*. 

Note that this is *not*  the same as the standard deviation of the distribution of $X$ (which we know should be 1).

In [ ]:
def sample_std_dev(data):
    return numpy.std(data)

print(sample_std_dev(means))

We can also estimate the variance of the standard deviation estimator with the bootstrapping method. Again we plot a histogram of the std deviation across the bootstrap samples.

In [ ]:
stddevs = []
for s in range(nsamples) : 
    bootstrap = random.choices(observations,k=50)
    stddevs.append(sample_std_dev(bootstrap))

plt.hist(stddevs,bins=100)
plt.show()
print(sample_mean(stddevs))

You should be careful when dealing with small lists of observations since theres a limit to the number of unique bootstrap samples that can be obtained. In our example, where we had 50 in our sample this number is 101!/(50!)(49!) so you don't need to worry.

Finally, bootstrap estimates are only asymptotically consistent in certain circumstances and so should be used with caution.

## Method of moments and bootstrap

Now let's apply what we just learned to our method of moments estimators for the gaussian $\mu$ and $\sigma$ parameters.  

For each value of $n$, we can estimate the variance (or standard deviation) of the distribution of our estimator using bootstrap samples. 

In [ ]:
def bootstrap_mu(samples):
    nsamples = len(samples)
    mus = []
    for s in range(nsamples) : 
      bootstrap = random.choices(samples,k=nsamples)
      mus.append(numpy.mean(bootstrap))
    return numpy.std(mus)

def bootstrap_sigma(samples):
    nsamples = len(samples)
    sigmas = []
    for s in range(nsamples) : 
      bootstrap = random.choices(samples,k=nsamples)
      sigmas.append(numpy.std(bootstrap))
    return numpy.std(sigmas)

ns_sets = range(10,1000,50)

mu_hats = []
sigma_hats = []
mu_hats_stddev = []
sigma_hats_stddev = []

for nsamples in ns_sets:
    toys = norm.rvs(mu_true,sigma_true,size=nsamples)
    mu_hats.append(numpy.mean(toys))
    mu_hats_stddev.append(bootstrap_mu(toys))
    sigma_hats.append(numpy.std(toys))
    sigma_hats_stddev.append(bootstrap_sigma(toys))

In [ ]:
plt.plot(ns_sets,mu_hats,color='blue',marker="o",label="$\hat{\mu}$")
plt.plot(ns_sets,sigma_hats,color='red',marker="o",label="$\hat{\sigma}$")
plt.fill_between(ns_sets, [m-e for m,e in zip(mu_hats,mu_hats_stddev)], [m+e for m,e in zip(mu_hats,mu_hats_stddev)])
plt.fill_between(ns_sets, [s-e for s,e in zip(sigma_hats,sigma_hats_stddev)], [s+e for s,e in zip(sigma_hats,sigma_hats_stddev)])

plt.xlabel("# in sample")
plt.ylabel("$\hat{\mu}$ or $\hat{\sigma}$")

plt.plot([ns_sets[0],ns_sets[-1]],[mu_true,mu_true],color='black',linestyle="--")
plt.plot([ns_sets[0],ns_sets[-1]],[sigma_true,sigma_true],color='black',linestyle="--")

plt.legend()
plt.savefig("moments_estimator.pdf")
plt.show()

You can see that the standard deviation (the square root of the variance) from the bootstrapping gets narrower as the number in the sample increases. This makes sense since the more data we have, the more accurate we should be about the estimator.  

## Hubble constant 

Let's take a look at an example where Bootstrapping comes in handy in the "real world". Below is a plot of data from Hubble's paper ["A relation between distance and radial velocity among extra-galactic nebulae" (1929)](https://www.pnas.org/content/15/3/168), showing the radial velocity ($v$) of extra-galactic nebulae vs their distance from us ($d$). 

(I borrowed the data from this interesting [blog post](https://medium.datadriveninvestor.com/revisiting-hubbles-law-with-hacker-stats-in-python-9b56604916c1))

The data are saved in a `.csv` file with colums "name", "distances" (in Mpc), "velocities" (in km/s). We'll use the `pandas.read_csv` function to read in the data. Pandas is a popular tool to manipulate columnar data (either you've come across this already in the ML practical course or you will do very soon). 

In [ ]:
import pandas 
data = pandas.read_csv("hubble_data.csv")
data.head(24) # there are 24 rows in the data 

We don't need the names of course but we do need the other two columns. 

In [ ]:
import matplotlib.pyplot as plt
import random 
import numpy as np 


plt.scatter(data["distances"].tolist(),data["velocities"].tolist())
plt.xlabel("Distance (Mpc)")
plt.ylabel("Radial velocity (km/s)")
plt.show()

Remember that Hubble's law is that the two quantities plotted are related as 

$$
v = H_{0} d
$$

Where $H_{0}$ is the Hubble constant. 

We can use a simple linear regression to determine $H_{0}$ from the data. To estimate $H_0$, we optimise the least-squares, 

$$
s = \sum_{i=1}^{n} (v_{i} - H_{0}d_{i})^{2}
$$

where we assume the intercept is 0, the value of $H_0$ that minimises $s$ (i.e such that $\frac{ds}{dH_0}$=0) is given by, 

$$
H_{0} = \frac{\sum_i d_i v_i }{\sum_{i}d_i^{2} }
$$

In [ ]:
def H0(data): 
    sumdv = sum([d[0]*d[1] for d in data])
    sumdd = sum([d[0]*d[0] for d in data])
    
    return (sumdv)/(sumdd)

# let's put the data into a basic form
data_basic = [[d,v] for d,v in zip(data["distances"].tolist(),data["velocities"].tolist())]

H0_obs = H0(data_basic)
print(H0_obs)

plt.scatter(data["distances"].tolist(),data["velocities"].tolist())
plt.xlabel("Distance (Mpc)")
plt.ylabel("Radial velocity (km/s)")
plt.plot([0,2],[0,2*H0_obs],color='red')
plt.savefig("galaxies.pdf")
plt.show()

The value of $H_0$ here is about $7\times$ larger than the measured value today. This is due to the fact that the distances measured were around $7\times$ smaller than they should be. Let's ignore this though and carry on with these distances.

These data don't have any uncertainty associated to them, so we can't use typical error propagation to estimate an uncertainty on $H_{0}$. However, since each of these measurements are independent of one-another, we can estimate an uncertainty on $H_0$ using Bootstrapping. 

Remember, we select boostrap samples by randomly sampling from the data, with replacement, until we have a new sample with as many entries as the original data. 

In [ ]:
bootstrap_H0 = []
n = len(data_basic)
for i in range(100): 
  bootstrap = random.choices(data_basic,k=n)
  bootstrap_H0.append(H0(bootstrap))
  
plt.hist(bootstrap_H0)
plt.show()

print("H_0 = %g +/- %g (km/s)/(Mpc)"%(H0_obs,np.std(bootstrap_H0)))

We can see that the statistical uncertainty is around 10% in this case. However, you might already know that the current value is closer to ~70  (𝑘𝑚/𝑠)/(𝑀𝑝𝑐) . This is a crucial point of data analysis in that one can only get good results if both the data and model are good. In this case, we already mentioned that the distance measurements were a factor of ~7 off and we haven't accounted for that systematic error in our estimate.

Note that there are other forms of Bootstrapping, which may be more appropriate depending on what kind of situation we are dealing with. In the previous examples, we always assumed a fixed number of observations. However, in many scenarious, even the number of observations may be a random variable.

For example, if you were studying the energy spectrum of radioactive decay products, you might pick a fixed time-window to make your observations. In this case, the number of observations will be Poisson distributed. To reflect this, we can adjust our bootstrapping algorithm by attaching a weight  𝑤  to each selection  𝑤∼Poisson(𝜆=1) . If we have 100 entries in the original dataset, we make 100 selections but a single selection may have a weight of  0  (i.e its not included in our sample) or  2  (so its included twice in the sample) or  3,4,5... . The total number therefore may not be 100 in the bootstrap sample.